Q1. In order to predict house price based on several characteristics, such as location, square footage,
number of bedrooms, etc., you are developing an SVM regression model. Which regression metric in this
situation would be the best to employ?

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
df=pd.read_csv("Bengaluru_House_Data.csv")

In [44]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13319 entries, 0 to 13318
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     9999 non-null   object 
 1   availability  9999 non-null   object 
 2   location      9998 non-null   object 
 3   size          9986 non-null   object 
 4   society       5844 non-null   object 
 5   total_sqft    9999 non-null   object 
 6   bath          9937 non-null   float64
 7   balcony       9519 non-null   float64
 8   price         9999 non-null   float64
dtypes: float64(3), object(6)
memory usage: 936.6+ KB


In [46]:
df.isna().sum()

area_type       3320
availability    3320
location        3321
size            3333
society         7475
total_sqft      3320
bath            3382
balcony         3800
price           3320
dtype: int64

In [47]:
df.dropna(inplace=True)

In [48]:
df

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
5,Super built-up Area,Ready To Move,Whitefield,2 BHK,DuenaTa,1170,2.0,1.0,38.00
11,Plot Area,Ready To Move,Whitefield,4 Bedroom,Prrry M,2785,5.0,3.0,295.00
...,...,...,...,...,...,...,...,...,...
9987,Super built-up Area,Ready To Move,Sarjapur Road,2 BHK,Maantr,1314,2.0,2.0,110.00
9989,Built-up Area,Ready To Move,Virudhu Nagar,2 BHK,AmthaIn,1130,2.0,2.0,68.00
9994,Super built-up Area,Ready To Move,Marathahalli,3 BHK,Valdsic,1550,3.0,2.0,85.00
9995,Super built-up Area,Ready To Move,Hoodi,3 BHK,SMaxyay,1715,2.0,2.0,100.00


In [49]:
for index,rows in df.iterrows():
    if rows["total_sqft"].isdigit():
        float(rows["total_sqft"])
    else:
        df.drop(index=index,inplace=True)     

In [50]:
df["total_sqft"]=df["total_sqft"].astype(float)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5419 entries, 0 to 9996
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     5419 non-null   object 
 1   availability  5419 non-null   object 
 2   location      5419 non-null   object 
 3   size          5419 non-null   object 
 4   society       5419 non-null   object 
 5   total_sqft    5419 non-null   float64
 6   bath          5419 non-null   float64
 7   balcony       5419 non-null   float64
 8   price         5419 non-null   float64
dtypes: float64(4), object(5)
memory usage: 423.4+ KB


In [52]:
df.describe()

,total_sqft,bath,balcony,price
count,5419.000000,5419.000000,5419.000000,5419.000000
mean,1501.100757,2.447684,1.663960,96.153032
std,991.635794,0.865004,0.754594,108.301648
min,11.000000,1.000000,0.000000,10.000000
25%,1115.000000,2.000000,1.000000,49.350000
50%,1305.000000,2.000000,2.000000,69.890000
75%,1645.000000,3.000000,2.000000,104.000000
max,36000.000000,9.000000,3.000000,2700.000000


In [53]:
from sklearn.preprocessing import OneHotEncoder

In [54]:
one=OneHotEncoder()

In [55]:
encoded=one.fit_transform(df[['area_type','availability','location','size','society']])

In [56]:
encoded

<5419x2823 sparse matrix of type '<class 'numpy.float64'>'
	with 27095 stored elements in Compressed Sparse Row format>

In [57]:
encoded_df=pd.DataFrame(data=encoded.toarray(),columns=one.get_feature_names_out())

In [58]:
encoded_df

,area_type_Built-up Area,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,availability_15-Aug,availability_15-Dec,availability_15-Jun,availability_16-Dec,availability_16-Mar,availability_16-Oct,...,society_WishiVa,society_WoarkEm,society_Xeitaa,society_YCnce R,society_Yaenti,society_ZeodsWo,society_Zonce E,society_Zostaa,society_i1odsne,society_i1rtsCo
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5414,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5415,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5416,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5417,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
final=pd.concat([df,encoded_df],axis=1)

In [60]:
final.drop(columns=['area_type','availability','location','size','society'],inplace=True)

In [61]:
final.dropna(inplace=True)

In [62]:
final.head()

,total_sqft,bath,balcony,price,area_type_Built-up Area,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,availability_15-Aug,availability_15-Dec,...,society_WishiVa,society_WoarkEm,society_Xeitaa,society_YCnce R,society_Yaenti,society_ZeodsWo,society_Zonce E,society_Zostaa,society_i1odsne,society_i1rtsCo
0,1056.0,2.0,1.0,39.07,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2600.0,5.0,3.0,120.00,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1521.0,3.0,1.0,95.00,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1170.0,2.0,1.0,38.00,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2785.0,5.0,3.0,295.00,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
final.columns

Index(['total_sqft', 'bath', 'balcony', 'price', 'area_type_Built-up  Area',
       'area_type_Carpet  Area', 'area_type_Plot  Area',
       'area_type_Super built-up  Area', 'availability_15-Aug',
       'availability_15-Dec',
       ...
       'society_WishiVa', 'society_WoarkEm', 'society_Xeitaa ',
       'society_YCnce R', 'society_Yaenti ', 'society_ZeodsWo',
       'society_Zonce E', 'society_Zostaa ', 'society_i1odsne',
       'society_i1rtsCo'],
      dtype='object', length=2827)

In [64]:
#seperate dependent and independent features
X=final.drop(columns=['price'])
y=final['price']

In [65]:
from sklearn.model_selection import train_test_split

Q2. You have built an SVM regression model and are trying to decide between using MSE or R-squared as
your evaluation metric. Which metric would be more appropriate if your goal is to predict the actual price
of a house as accurately as possible?

In [66]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [67]:
X_train.shape,X_test.shape

((2208, 2826), (737, 2826))

In [68]:
y_train.shape,y_test.shape

((2208,), (737,))

In [69]:
from sklearn.svm import SVR

In [70]:
svr=SVR()
svr.fit(X_train,y_train)

SVR()

In [71]:
y_pred=svr.predict(X_test)

In [72]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

0.24721598952056922
12414.132598429158
29.5821154603062
111.41872642616751


The best appropriate metric is r2 score

Q3. You have a dataset with a significant number of outliers and are trying to select an appropriate
regression metric to use with your SVM model. Which metric would be the most appropriate in this
scenario?

On this senario mean sqared eror is best 

Q4. You have built an SVM regression model using a polynomial kernel and are trying to select the best
metric to evaluate its performance. You have calculated both MSE and RMSE and found that both values
are very close. Which metric should you choose to use in this case?

In [32]:
svr=SVR(kernel='poly')

In [33]:
svr.fit(X_train,y_train)

SVR(kernel='poly')

In [34]:
y_pred=svr.predict(X_test)

In [35]:
print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

-0.043649169520259434
17210.778917067586
49.44525627983533
131.18985828587356


Q5. You are comparing the performance of different SVM regression models using different kernels (linear,
polynomial, and RBF) and are trying to select the best evaluation metric. Which metric would be most
appropriate if your goal is to measure how well the model explains the variance in the target variable?

In [36]:
svr=SVR(kernel='linear')

In [37]:
svr.fit(X_train,y_train)

SVR(kernel='linear')

In [38]:
y_pred=svr.predict(X_test)

In [39]:
print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

0.5132449008033542
8027.059900674936
26.885906491941366
89.59386084255402


In [75]:
svr=SVR(kernel='rbf')

In [76]:
svr.fit(X_train,y_train)

SVR()

In [77]:
y_pred=svr.predict(X_test)

In [78]:
print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

0.24721598952056922
12414.132598429158
29.5821154603062
111.41872642616751


This best model for this senario is polynomial and mean_absolute_error is best metric for the evaluation